In [1]:
import matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import cv2
import tkinter as tk
from utils import *
from models import *
from tkinter import filedialog
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import keras as k
from keras.models import Sequential ,Model
from keras.layers import Conv2D, MaxPooling2D,AveragePooling3D,AveragePooling2D,MaxPooling3D,Conv3D
from keras.layers import Activation, Dropout, Flatten, Dense ,Input
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing import image
from keras import applications

In [3]:
googlenet = makegooglenet()
lenet = makelenet()

In [4]:
path = ""
numpyimage = 0
i=0
canvas = 0
canvas2 = 0
Textbox=0
Textbox2=0
slicenum =20
indexnodule = 0
indexslice = 20
nodules=[]
malig = []


In [5]:
def getnodules(crops,googlenet):
    count1=0
    count0=0
    global nodules
    for crop in crops:
        cc = np.expand_dims(crop,0)
        cc = np.expand_dims(cc,-1)
        if np.argmax(googlenet.predict(cc))==1:
            nodules.append(crop)
            count1= count1+1
        else:
            count0=count0+1
    return nodules

def next():
    global i
    global canvas
    if i< numpyimage.shape[0]:
        i= i+1
    f = Figure(figsize=(4,4), dpi=100)
    a= f.add_subplot(1,1,1)
    a.imshow(numpyimage[i],cmap='gray')
    canvas = FigureCanvasTkAgg(f,gui,)
    canvas.get_tk_widget().grid(row=1,column=0)
    
def prev():
    global i
    global canvas
    if i >0:
        i= i-1
    f = Figure(figsize=(4,4), dpi=100)
    a= f.add_subplot(1,1,1)
    a.imshow(numpyimage[i],cmap='gray')
    canvas = FigureCanvasTkAgg(f,gui,)
    canvas.get_tk_widget().grid(row=1,column=0)
    
def nextnod():
    global indexnodule
    global canvas2
    global slicenum
    if indexnodule< len(nodules)-1:
        indexnodule= indexnodule+1
    f = Figure(figsize=(4,4), dpi=100)
    a= f.add_subplot(1,1,1)
    a.imshow(nodules[indexnodule][slicenum],cmap='gray')
    canvas2 = FigureCanvasTkAgg(f,gui,)
    canvas2.get_tk_widget().grid(row=1,column=1)
    Textbox = tk.Text(gui, height=2, width=20)
    Textbox.grid(row=0,column=2)
    Textbox.insert(tk.END, "malignancy : "+ str(malig[indexnodule]))

def prevnod():
    global indexnodule
    global canvas2
    global slicenum
    if indexnodule>0:
        indexnodule= indexnodule-1
    f = Figure(figsize=(4,4), dpi=100)
    a= f.add_subplot(1,1,1)
    a.imshow(nodules[indexnodule][slicenum],cmap='gray')
    canvas2 = FigureCanvasTkAgg(f,gui,)
    canvas2.get_tk_widget().grid(row=1,column=1)
    Textbox = tk.Text(gui, height=2, width=20)
    Textbox.grid(row=0,column=2)
    Textbox.insert(tk.END, "malignancy : "+ str(malig[indexnodule]))
    
def nextslice():
    global indexnodule
    global indexslice
    global canvas2
    if indexslice< 63:
        indexslice= indexslice+1
    f = Figure(figsize=(4,4), dpi=100)
    a= f.add_subplot(1,1,1)
    a.imshow(nodules[indexnodule][indexslice],cmap='gray')
    canvas2 = FigureCanvasTkAgg(f,gui,)
    canvas2.get_tk_widget().grid(row=1,column=1)
    
def prevslice():
    global indexnodule
    global indexslice
    global canvas2
    if indexslice>0:
        indexslice= indexslice-1
    f = Figure(figsize=(4,4), dpi=100)
    a= f.add_subplot(1,1,1)
    a.imshow(nodules[indexnodule][indexslice],cmap='gray')
    canvas2 = FigureCanvasTkAgg(f,gui,)
    canvas2.get_tk_widget().grid(row=1,column=1)



def browsefunc():
    filename = filedialog.askopenfilename()
    global path
    global numpyimage
    path = filename
    numpyimage,b,x = load_itk_image(path)
    
    f = Figure(figsize=(40,40), dpi=10)
    a= f.add_subplot(1,1,1)
    a.imshow(numpyimage[0] ,cmap='gray')
    canvas = FigureCanvasTkAgg(f,gui,)
    canvas.get_tk_widget().grid(row=1,column=0)
    
def analyze():
    global numpyimage
    global googlenet
    global nodules
    global indexslice
    global malig
    crops = getcrops(numpyimage)
    nodules = getnodules(crops,googlenet)
    
    for nodule in  nodules:
        cc = np.expand_dims(nodule,0)
        mnum = np.argmax(lenet.predict(cc))
        malig.append(mnum)
    f = Figure(figsize=(40,40), dpi=10)
    a= f.add_subplot(1,1,1)
    a.imshow(nodules[0][indexslice] ,cmap='gray')
    canvas2 = FigureCanvasTkAgg(f,gui,)
    canvas2.get_tk_widget().grid(row=1,column=1)
    
    Textbox = tk.Text(gui, height=2, width=20)
    Textbox.grid(row=0,column=2)
    Textbox.insert(tk.END, "malignancy : "+ str(malig[indexnodule]))
    
    Textbox2 = tk.Text(gui, height=2, width=20)
    Textbox2.grid(row=2,column=2)
    Textbox2.insert(tk.END, "nodule count : "+ str(len(nodules)))

In [ ]:
global canvas
global canvas2
global Textbox
global Textbox2
global i



gui = tk.Tk()
gui.minsize(800,500)
gui.configure(background='gray')

f = Figure(figsize=(40,40), dpi=10)
a= f.add_subplot(1,1,1)
a.imshow(np.zeros((5,5)) ,cmap='gray')
canvas = FigureCanvasTkAgg(f,gui,)
canvas.get_tk_widget().grid(row=1,column=0)


Textbox = tk.Text(gui, height=2, width=20)
Textbox.grid(row=0,column=2)
Textbox.insert(tk.END, "malignancy : ")


Textbox2 = tk.Text(gui, height=2, width=20)
Textbox2.grid(row=2,column=2)
Textbox2.insert(tk.END, "nodule count : ")


f = Figure(figsize=(40,40), dpi=10)
a= f.add_subplot(1,1,1)
a.imshow(np.zeros((5,5)) ,cmap='gray')
canvas2 = FigureCanvasTkAgg(f,gui,)
canvas2.get_tk_widget().grid(row=1,column=1)


bbrowse = tk.Button(gui , text="Browse" ,command=browsefunc )
bbrowse.grid(row=0,column=0)

anayliz = tk.Button(gui , text="Analyze" ,command=analyze )
anayliz.grid(row=0,column=1)


bnextnod = tk.Button(gui , text="Next Nodule" ,command=nextnod )
bnextnod.grid(row=2,column=1)
bprevnod = tk.Button(gui , text="Previous Nodule" ,command=prevnod )
bprevnod.grid(row=3,column=1)

bnextslice = tk.Button(gui , text="Next slice" ,command=nextslice )
bnextslice.grid(row=1,column=2)
bprevslice = tk.Button(gui , text="Previous slice" ,command=prevslice )
bprevslice.grid(row=1,column=3)


bnext = tk.Button(gui , text="Next" ,command=next )
bnext.grid(row=2,column=0)
bprev = tk.Button(gui , text="Previous" ,command=prev )
bprev.grid(row=3,column=0)


gui.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\tkinter\__init__.py", line 1558, in __call__
    return self.func(*args)
  File "<ipython-input-5-4959f1f8495b>", line 98, in browsefunc
    numpyimage,b,x = load_itk_image(path)
  File "C:\Users\Abdullah\Documents\grad\GUI\utils.py", line 4, in load_itk_image
    itkimage = sitk.ReadImage(filename)
  File "C:\Users\Abdullah\Anaconda3\envs\tensorflowEnv\lib\site-packages\SimpleITK\SimpleITK.py", line 8614, in ReadImage
    return _SimpleITK.ReadImage(*args)
RuntimeError: Exception thrown in SimpleITK ReadImage: C:\d\VS14-Win64-pkg\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:89:
sitk::ERROR: The file "" does not exist.
